In [1]:
install.packages("disk.frame", repo="https://cran.rstudio.com/")

library(forcats)
install.packages('dplyr')
library(MASS)
library(randomForest)
library(caret)
library(reticulate)
library(dplyr)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()
#inputData = read.csv(file = 'chunk4-vehicles.csv', header = TRUE, sep = ',')
#summary(inputData)


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘ellipsis’, ‘pillar’, ‘glue’, ‘lifecycle’, ‘tibble’, ‘tidyselect’, ‘vctrs’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘ggplot2’

The following object is masked from ‘package:randomForest’:

    margin


Attaching package: ‘dplyr’

The following object is masked from ‘package:randomForest’:

    combine

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [1]:
data <- get(load(file = "ScaledNumericData.RData"))
#head(data)
#data <- as.dataframe(data)
#sapply(data, class)
str(data)
summary(data)
dim(data)

'data.frame':	556298 obs. of  76 variables:
 $ ID                          : chr  "d5db547ae8e87d127ce0d4e51447befa5ff5012b_2019-08-04T02:50:00Z" "8ba1ffcd025f4002f0a94b9e5414a8868b40604f_2018-01-05T00:52:00Z" "75ccf97f6dbc795bd0bfc34067ab6b1a3d7e9325_2017-05-01T05:00:00Z" "ce3585171e5bf2dbaa366ebb9157cdbd790bcd66_2017-02-10T00:34:00Z" ...
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                         : num  0.331 0.29 0.537 0.824 0.331 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  -0.26 -0.26 -0.26 -0.26 -0.26 ...
 $ AVPU_1                      : num  -0.1489 0.0287 0.0111 -0.1489 -0.1489 ...
 $ AVPU_1_DT_TM                : num  0.0307 -0.02648 -0.02648 -0.00368 0.01594 ...
 $ GCS_1                       : num  0.060655 0.000335 0.003378 0.256747 0.003378 ...
 $ GCS_1_DT_TM                 : num  -0.06299 -0.06299 -0.06299 -0.00181 -0.06299 ...
 $ RR_1                        : num  -0.4047 -0.2827 0.5204 -0.0963 -0.4047 ...
 $ RR_1_DT_TM                  : n

      ID            TRIAGE_CATEGORY      AGE          ADMISSION_WITHIN_48_HOURS
 Length:556298      Min.   :1.000   Min.   :-1.7236   Min.   :-0.2595          
 Class :character   1st Qu.:2.000   1st Qu.:-0.7373   1st Qu.:-0.2595          
 Mode  :character   Median :3.000   Median :-0.1208   Median :-0.2595          
                    Mean   :3.156   Mean   : 0.0000   Mean   : 0.0000          
                    3rd Qu.:4.000   3rd Qu.: 0.7832   3rd Qu.:-0.2595          
                    Max.   :5.000   Max.   : 2.6736   Max.   : 3.8528          
     AVPU_1           AVPU_1_DT_TM           GCS_1           
 Min.   :-0.148948   Min.   :-31.69604   Min.   :-22.484970  
 1st Qu.:-0.148948   1st Qu.: -0.02648   1st Qu.:  0.003378  
 Median :-0.148948   Median : -0.02461   Median :  0.143044  
 Mean   : 0.000000   Mean   :  0.00000   Mean   :  0.000000  
 3rd Qu.: 0.000915   3rd Qu.: -0.01004   3rd Qu.:  0.256747  
 Max.   :17.635976   Max.   :235.40949   Max.   :  0.256747  
  GCS_

[1] 556298     76

In [3]:
ED_data <- data[, -c(1, 4, 7, 9,10, 64)]
str(ED_data)

ED_data$GENDER <- as.numeric(ED_data$GENDER)
ED_data$MODEL_OF_CARE <- as.numeric(ED_data$MODEL_OF_CARE)


# create Age Group variable for mean imputation
ED_data$AGE_GROUP <- data$AGE %/% 10
ED_data$RR_grouped <- (data$RR_1 < 10)
ED_data$BP_grouped <- (data$SBP_1 < 80)
ED_data$HR_grouped <- (data$HEART_RATE_1 < 50 | data$HEART_RATE_1 > 150)
ED_data$GCS_critical <- (data$GCS_1 < 9 )
ED_data$GCS_unresponsive <- (data$GCS_1 > 9 & data$GCS_1 < 13)
ED_data[,c(1, 93,94, 95, 96, 97)] <- lapply(ED_data[,c(1, 93,94, 95, 96, 97)], as.numeric)
str(ED_data)

'data.frame':	556298 obs. of  91 variables:
 $ TRIAGE_CATEGORY                           : Factor w/ 5 levels "1","2","3","4",..: 3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                                       : num  50 49 55 62 50 75 46 69 70 86 ...
 $ GENDER                                    : Factor w/ 2 levels "Female","Male": 1 2 2 2 2 2 1 2 2 2 ...
 $ ADMISSION_WITHIN_48_HOURS                 : num  0 0 0 0 0 0 0 0 0 0 ...
 $ MODEL_OF_CARE                             : Factor w/ 22 levels "#NOF Pathway",..: 3 11 11 NA 11 11 11 11 3 11 ...
 $ AVPU_1                                    : num  1 1.04 1.04 1 1 ...
 $ AVPU_1_DT_TM                              : num  18360 NA NA 7320 13620 ...
 $ AVPU_2                                    : num  1 NA NA 1 1 1 1 1 1 1 ...
 $ AVPU_2_DT_TM                              : num  25620 NA NA 11340 20400 ...
 $ GCS_1                                     : num  14.9 14.9 14.9 15 14.9 ...
 $ GCS_1_DT_TM                               : num  NA NA NA 7320 NA ..

In [4]:
#triageData <- triageData[-c(1,2, 4,7)]#remove dependent and descriptor variables
# function for mean imputation
impute.mean <- function(x) replace(x, is.na(x), mean(x, na.rm = TRUE))

# function for replacing with zero
replace.zero <- function(x) replace(x, is.na(x), 0)

# subset first observations
Numeric_cols <- colnames(ED_data %>% select_if(is.numeric))
First_obs_cols <- Numeric_cols[endsWith(Numeric_cols, '_1')]
Second_obs_cols <- Numeric_cols[endsWith(Numeric_cols, '_2')]
Body_measure_cols <- Numeric_cols[Numeric_cols %in% c("PATIENT_HEIGHT","PATIENT_WEIGHT","MODEL_OF_CARE")]
#Binary_cols <- Numeric_cols[Numeric_cols %in% c("PATIENT_HEIGHT","PATIENT_WEIGHT")]

# group data by GENDER and AGE GROUP then impute the group mean for missing values
ED_data = ED_data %>% group_by(AGE_GROUP , GENDER) %>%
  mutate_at(First_obs_cols, impute.mean) %>%  # initial observations
  #mutate_at(Second_obs_cols, impute.mean) %>%  # second observations
  mutate_at(Body_measure_cols, impute.mean) # body measurements
#mutate_at(Binary_cols, replace.zero)  # binary variables replace with mode (0 for all)
str(ED_data)


tibble [556,298 × 97] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ TRIAGE_CATEGORY                           : num [1:556298] 3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                                       : num [1:556298] 50 49 55 62 50 75 46 69 70 86 ...
 $ GENDER                                    : num [1:556298] 1 2 2 2 2 2 1 2 2 2 ...
 $ ADMISSION_WITHIN_48_HOURS                 : num [1:556298] 0 0 0 0 0 0 0 0 0 0 ...
 $ MODEL_OF_CARE                             : num [1:556298] 3 11 11 11.3 11 ...
 $ AVPU_1                                    : num [1:556298] 1 1.04 1.04 1 1 ...
 $ AVPU_1_DT_TM                              : num [1:556298] 18360 NA NA 7320 13620 ...
 $ AVPU_2                                    : num [1:556298] 1 NA NA 1 1 1 1 1 1 1 ...
 $ AVPU_2_DT_TM                              : num [1:556298] 25620 NA NA 11340 20400 ...
 $ GCS_1                                     : num [1:556298] 14.9 14.9 14.9 15 14.9 ...
 $ GCS_1_DT_TM                               : num [1:556298] NA 

In [5]:
library(magrittr)
library(dplyr)
#### create reusable subsets of data ####
# create lists of variable types to use for indexing
Numeric_cols <- colnames(ED_data %>% select_if(is.numeric)) 
# create drop list
Time_cols <- Numeric_cols[endsWith(Numeric_cols, 'DT_TM')] 
Second_obs_cols <- Numeric_cols[endsWith(Numeric_cols, '_2')]
drop_from_keras <- c(Time_cols, Second_obs_cols)
#drop_from_keras <- c(Time_cols) 
# create a subset of numeric values
Numeric_data <- ED_data[,Numeric_cols]
# remove second observations and time observations
Numeric_data <- Numeric_data[, !colnames(Numeric_data) %in% drop_from_keras]
#response <- as.data.frame(data$TRIAGE_CATEGORY)
#Numeric_plus_response <- cbind(response, Numeric_data) 
Numeric_data[,1] <- lapply(Numeric_data[,1], as.factor)
str(Numeric_data)
summary(Numeric_data)

tibble [556,298 × 61] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ TRIAGE_CATEGORY                           : Factor w/ 5 levels "1","2","3","4",..: 3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                                       : num [1:556298] 50 49 55 62 50 75 46 69 70 86 ...
 $ GENDER                                    : num [1:556298] 1 2 2 2 2 2 1 2 2 2 ...
 $ ADMISSION_WITHIN_48_HOURS                 : num [1:556298] 0 0 0 0 0 0 0 0 0 0 ...
 $ MODEL_OF_CARE                             : num [1:556298] 3 11 11 11.3 11 ...
 $ AVPU_1                                    : num [1:556298] 1 1.04 1.04 1 1 ...
 $ GCS_1                                     : num [1:556298] 14.9 14.9 14.9 15 14.9 ...
 $ RR_1                                      : num [1:556298] 18 18.8 24 20 18 ...
 $ O2SATS_1                                  : num [1:556298] 100 98.1 91 93 99 ...
 $ FIO2_1                                    : num [1:556298] 35.8 43 39.6 710.4 39.6 ...
 $ PULSE_1                                   : num 

 TRIAGE_CATEGORY      AGE             GENDER      ADMISSION_WITHIN_48_HOURS
 1:  8469        Min.   :  0.00   Min.   :1.000   Min.   :0.00000          
 2:143930        1st Qu.: 24.00   1st Qu.:1.000   1st Qu.:0.00000          
 3:198847        Median : 39.00   Median :2.000   Median :0.00000          
 4:162678        Mean   : 41.94   Mean   :1.504   Mean   :0.06311          
 5: 42374        3rd Qu.: 61.00   3rd Qu.:2.000   3rd Qu.:0.00000          
                 Max.   :107.00   Max.   :2.000   Max.   :1.00000          
 MODEL_OF_CARE       AVPU_1          GCS_1            RR_1       
 Min.   : 1.00   Min.   :1.000   Min.   : 3.00   Min.   :  0.00  
 1st Qu.:11.00   1st Qu.:1.000   1st Qu.:14.87   1st Qu.: 18.00  
 Median :11.00   Median :1.000   Median :14.94   Median : 18.82  
 Mean   :11.48   Mean   :1.034   Mean   :14.86   Mean   : 20.62  
 3rd Qu.:11.00   3rd Qu.:1.034   3rd Qu.:15.00   3rd Qu.: 20.00  
 Max.   :22.00   Max.   :5.000   Max.   :15.00   Max.   :249.00  
    O2

In [6]:
library(caret)
set.seed(100) # set seed to replicate results
trainingIndex <- sample(1:nrow(Numeric_data), 0.7*nrow(Numeric_data)) # indices for 70% training data
trainingData <- Numeric_data[trainingIndex, ] # training data
testData <- Numeric_data[-trainingIndex, ] # test data

names(trainingData)[names(trainingData) == "PRESENTING_GROUPMental Health"] <- "PRESENTING_GROUP_MentalHealth"
names(trainingData)[names(trainingData) == "PRESENTING_GROUPObstetrics and Gynaecology"] <- "PRESENTING_GROUP_Obstetrics_Gynaecology"
names(trainingData)[names(trainingData) == "PRESENTING_GROUPENT, face"] <- "PRESENTING_GROUP_ENTface"
names(testData)[names(testData) == "PRESENTING_GROUPMental Health"] <- "PRESENTING_GROUP_MentalHealth"
names(testData)[names(testData) == "PRESENTING_GROUPObstetrics and Gynaecology"] <- "PRESENTING_GROUP_Obstetrics_Gynaecology"
names(testData)[names(testData) == "PRESENTING_GROUPENT, face"] <- "PRESENTING_GROUP_ENTface"

summary(trainingData)
summary(testData)
str(trainingData)
str(testData)

 TRIAGE_CATEGORY      AGE             GENDER      ADMISSION_WITHIN_48_HOURS
 1:  5948        Min.   :  0.00   Min.   :1.000   Min.   :0.00000          
 2:100592        1st Qu.: 24.00   1st Qu.:1.000   1st Qu.:0.00000          
 3:139227        Median : 39.00   Median :2.000   Median :0.00000          
 4:114047        Mean   : 41.95   Mean   :1.502   Mean   :0.06288          
 5: 29594        3rd Qu.: 61.00   3rd Qu.:2.000   3rd Qu.:0.00000          
                 Max.   :106.00   Max.   :2.000   Max.   :1.00000          
 MODEL_OF_CARE       AVPU_1          GCS_1            RR_1       
 Min.   : 1.00   Min.   :1.000   Min.   : 3.00   Min.   :  0.00  
 1st Qu.:11.00   1st Qu.:1.000   1st Qu.:14.87   1st Qu.: 18.00  
 Median :11.00   Median :1.000   Median :14.94   Median : 18.82  
 Mean   :11.47   Mean   :1.034   Mean   :14.86   Mean   : 20.62  
 3rd Qu.:11.00   3rd Qu.:1.034   3rd Qu.:15.00   3rd Qu.: 20.00  
 Max.   :22.00   Max.   :5.000   Max.   :15.00   Max.   :249.00  
    O2

 TRIAGE_CATEGORY      AGE             GENDER      ADMISSION_WITHIN_48_HOURS
 1: 2521         Min.   :  1.00   Min.   :1.000   Min.   :0.00000          
 2:43338         1st Qu.: 24.00   1st Qu.:1.000   1st Qu.:0.00000          
 3:59620         Median : 39.00   Median :2.000   Median :0.00000          
 4:48631         Mean   : 41.93   Mean   :1.506   Mean   :0.06366          
 5:12780         3rd Qu.: 61.00   3rd Qu.:2.000   3rd Qu.:0.00000          
                 Max.   :107.00   Max.   :2.000   Max.   :1.00000          
 MODEL_OF_CARE      AVPU_1          GCS_1            RR_1       
 Min.   : 1.0   Min.   :1.000   Min.   : 3.00   Min.   :  1.00  
 1st Qu.:11.0   1st Qu.:1.000   1st Qu.:14.87   1st Qu.: 18.00  
 Median :11.0   Median :1.000   Median :14.94   Median : 18.82  
 Mean   :11.5   Mean   :1.033   Mean   :14.87   Mean   : 20.64  
 3rd Qu.:11.0   3rd Qu.:1.034   3rd Qu.:15.00   3rd Qu.: 20.00  
 Max.   :22.0   Max.   :5.000   Max.   :15.00   Max.   :214.00  
    O2SATS_1 

tibble [389,408 × 61] (S3: grouped_df/tbl_df/tbl/data.frame)
 $ TRIAGE_CATEGORY                        : Factor w/ 5 levels "1","2","3","4",..: 2 3 2 4 3 3 3 4 4 3 ...
 $ AGE                                    : num [1:389408] 62 47 63 36 64 2 47 47 79 83 ...
 $ GENDER                                 : num [1:389408] 2 1 1 2 1 2 2 2 1 1 ...
 $ ADMISSION_WITHIN_48_HOURS              : num [1:389408] 0 0 0 0 0 0 0 0 0 0 ...
 $ MODEL_OF_CARE                          : num [1:389408] 7 11.3 11 12.4 11 ...
 $ AVPU_1                                 : num [1:389408] 1 1 1 1 1.02 ...
 $ GCS_1                                  : num [1:389408] 14.8 15 14.9 14.9 14.9 ...
 $ RR_1                                   : num [1:389408] 17 18 20 18 20 ...
 $ O2SATS_1                               : num [1:389408] 100 100 96 96 99 100 98 99 97 97 ...
 $ FIO2_1                                 : num [1:389408] 710.4 41.6 35.7 40.8 35.7 ...
 $ PULSE_1                                : num [1:389408] 64 70 64 

In [ ]:
set.seed(1234)# can be any number 

rf.model1 <- randomForest(TRIAGE_CATEGORY ~., data = trainingData, na.action = na.omit, proximity = FALSE, importance = TRUE)
rf.model1

In [ ]:
# Fine tuning parameters of Random Forest model
rf.model2 <- randomForest(TRIAGE_CATEGORY ~ ., data = trainingData, ntree = 500, mtry = 6, proximity = FALSE, importance = TRUE, na.action = na.omit)
rf.model2

In [ ]:
trainset = trainingData
testset <- testData
predtrain = predict(rf.model2, trainset, type = "class")
table(predtrain, trainset$TRIAGE_CATEGORY)

str(trainset)

trainset = trainingData
testset <- testData
predvalid <- predict(rf.model2, testset, type = "class")
mean(predvalid == testset$TRIAGE_CATEGORY)
table(predvalid, testset$TRIAGE_CATEGORY)

In [ ]:
importance(rf.model2, descending)        
varImpPlot(rf.model2)  

In [ ]:
# Using For loop to identify the right mtry for model
a=c()
i=5
for (i in 3:8) {
  model3 <- randomForest(TRIAGE_CATEGORY ~ ., data = trainset, ntree = 500, mtry = i, importance = TRUE)
  predValid <- predict(model3, testset, type = "class")
  a[i-2] = mean(predValid == testset$TRIAGE_CATEGORY)
}
 
a
 
plot(3:8,a)